This is the Glob-style path to the subfolder containing niftis of interest
- For example, from the base_directory, */tissue_segment_z_scores will look for all subjects, all session folders within subjects, and then check the tissue_segment_z_scores folder. 


In [ ]:
path = '/Volumes/HowExp/datasets/02a_Corbetta_Stroke_Lesion/Study_Metadata/master_list.csv'
file_pattern = None #'*.nii.gz'
file_column = 'roi_path'

In [ ]:
from calvin_utils.file_utils.import_functions import GiiNiiFileImport
dv_df = GiiNiiFileImport(import_path=path, file_pattern=file_pattern, file_column=file_column, process_special_values=True).run()
dv_df

**Extract Subject ID From File Names**
Using the example filenames that have been printed above, please define a general string:
1) Preceding the subject ID.
2) Proceeding the subject ID. 

This Should Often Be Left Default

In [ ]:
preceding_id = '/sub-'
proceeding_id = '/roi'

In [ ]:
from calvin_utils.file_utils.import_functions import GiiNiiFileImport
dv_df = GiiNiiFileImport.splice_colnames(dv_df, preceding_id, proceeding_id)
dv_df

# Load Map to Check Damage Within

Import Region of Interest Masks

In [ ]:
base_directory = '/Volumes/HowExp/resources/atlases/atlases/MNI Lobes'
shared_glob_pattern = 'mni_Cerebellum.nii.gz'

In [ ]:
from calvin_utils.file_utils.import_functions import GiiNiiFileImport
iv_df = GiiNiiFileImport(import_path=base_directory, file_pattern=shared_glob_pattern).run()
iv_df

Extract Damage Scores Per Region of Interest

In [ ]:
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'

In [ ]:
from calvin_utils.nifti_utils.damage_score_utils import DamageScorer
damage_scorer = DamageScorer(mask_path, dv_df, iv_df)
dmg_df = damage_scorer.calculate_damage_scores('sum')
dmg_df = damage_scorer.sort_dataframes_by_index(dmg_df)
dmg_df

Rename Column in Damage DF

In [ ]:
dmg_df = dmg_df.rename_axis('path').reset_index()

In [ ]:
dmg_df = dmg_df.rename(columns={dmg_df.columns[0]: 'subject'})

In [ ]:
dmg_df.columns = [dmg_df.columns[0]] + [f"{col}_csf" for col in dmg_df.columns[1:]]

In [ ]:
dmg_df

Save the Results

In [ ]:
# out_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/metadata/master_list2.csv'

In [ ]:
# dmg_df.to_csv(out_path)

Save Results into Master CSV

In [ ]:
master_path = '/Volumes/HowExp/datasets/02a_Corbetta_Stroke_Lesion/Study_Metadata/master_list.csv'
sheet = None

In [ ]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
cal_palm = CalvinStatsmodelsPalm(input_csv_path=master_path, output_dir=None, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()
data_df

In [ ]:
# Perform an outer join on dmg_df and data_df using the 'path' column
merged_df = data_df.merge(dmg_df, on='subject', how='outer')
merged_df

In [ ]:
merged_df.to_csv(master_path, index=False)

In [ ]:
master_path

All done. Enjoy your analyses. 

--Calvin 